In [ ]:
#!pip install demoji
#!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=4e80dfc6330063aa5e8da164a6f8e37ffc51c56ffd7e7acba83edc77ef17d8a2
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import nltk
import spacy
import string
import re
import warnings
import demoji
import json


from nltk.corpus import stopwords
from nltk.collocations import *
from collections import Counter
from nltk.stem import porter
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC







nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('genesis')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))
stemmer = porter.PorterStemmer()

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

nlp = spacy.load("en_core_web_sm")

nltk.download('punkt')
nltk.download('brown')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [2]:
# Importing the dataset
url = 'https://raw.githubusercontent.com/shahrzadko/NLPDigitalMusic/main/Digital_Music_5.json?token=GHSAT0AAAAAAB6LOL42SKHPCCMIVYPOC4REY7AKE2Q'
df_raw = pd.read_json(url)
full_df = pd.DataFrame.from_records(df_raw['data'])

In [2]:
#full_df = pd.read_json("reviews_Digital_Music_5.json.gz", orient='records',lines=True)


In [3]:
#creating Sentiment column with overall rating
full_df['Sentiment'] = np.where((full_df['overall'] > 3), 'Positive', 
                          np.where((full_df['overall'] < 3), 'Negative', 'Neutral'))


In [4]:
#showing that the data is not balanced
full_df['Sentiment'].value_counts() 

Positive    158985
Neutral       6792
Negative      4004
Name: Sentiment, dtype: int64

In [5]:
#creating  'NewReview'  column using  'reviewText' and 'summary' columns
cols = ['reviewText','summary']
full_df['NewReview'] = full_df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
full_df['NewReview'] = full_df['NewReview'].astype(str)
full_df['review_length'] = full_df['NewReview'].apply(lambda x: len(x.split()))

In [6]:
music_df = full_df.drop(columns=['vote','image','reviewTime','style','reviewerName','unixReviewTime','summary','reviewText'])
#music_df = full_df.drop(columns=['reviewTime','reviewerName','unixReviewTime','summary','reviewText'])

## balance

In [7]:
#dropping rows if the review is not verified
music_df_balanced = music_df[music_df.verified == True]

#dropping rows if the reviewer generally gives very negative reviews
df_filtered = music_df_balanced.groupby('reviewerID').filter(lambda x: not((x['overall'].count() >= 10) and (x['overall'].mean() <= 2)))

#dropping rows if the review is too short
#df_filtered = df_filtered[df_filtered['review_length'] >= 6]


In [8]:
#checking the values of Sentiment to creat a sample data
df_filtered['Sentiment'].value_counts() 

Positive    141104
Neutral       5396
Negative      2093
Name: Sentiment, dtype: int64

In [9]:
import pandas as pd
import nltk
from nltk.corpus import wordnet


# Reduce the number of positive sentiment rows to 5000
df_pos = df_filtered[df_filtered['Sentiment'] == 'Positive'].sample(n=10000, random_state=42)

# Split the remaining dataset by sentiment
df_neutral = df_filtered[df_filtered['Sentiment'] == 'Neutral']
df_neg = df_filtered[df_filtered['Sentiment'] == 'Negative']

# Define function for synonym replacement
def synonym_replacement(text):
    words = nltk.word_tokenize(text)
    new_words = []
    for word in words:
        synonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if synonyms:
            new_word = synonyms[0]
        else:
            new_word = word
        new_words.append(new_word)
    new_text = ' '.join(new_words)
    return new_text

# Oversample minority classes using synonym replacement
desired_balance = 0.9
while len(df_neutral) / len(df_pos) < desired_balance or len(df_neg) / len(df_pos) < desired_balance:
    if len(df_neutral) / len(df_pos) < desired_balance:
        sample = df_neutral.sample(n=1)['NewReview'].iloc[0]
        new_review = synonym_replacement(sample)
        df_neutral = df_neutral.append({'NewReview': new_review, 'Sentiment': 'Neutral'}, ignore_index=True)
    if len(df_neg) / len(df_pos) < desired_balance:
        sample = df_neg.sample(n=1)['NewReview'].iloc[0]
        new_review = synonym_replacement(sample)
        df_neg = df_neg.append({'NewReview': new_review, 'Sentiment': 'Negative'}, ignore_index=True)




# Combine oversampled subsets back into a single dataframe
df_balanced = pd.concat([df_pos, df_neutral, df_neg], ignore_index=True)

# Shuffle the dataframe
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)


In [10]:
df_balanced['Sentiment'].value_counts() 

Positive    10000
Neutral      9000
Negative     9000
Name: Sentiment, dtype: int64

In [ ]:
#creating balanced data using equal number of each sentiment
#sample_size = min(df_filtered['Sentiment'].value_counts())
#df_balanced = df_filtered.groupby('Sentiment').apply(lambda x: x.sample(sample_size)).reset_index(drop=True)

In [ ]:
#df_balanced['Sentiment'].value_counts() 

In [11]:
df_balanced.head()

,overall,verified,reviewerID,asin,Sentiment,NewReview,review_length
0,5.0,True,ACQT14ZN20YQY,B009IQ2RSU,Positive,"I like this song, it is worth buying and spend...",25.0
1,5.0,True,A1NBZGHKMQFP41,B00ZF3SIDI,Positive,Loved this!! Five Stars,4.0
2,3.0,True,A3DCOGGBKMJ8RN,B00UB4WZSA,Neutral,good Three Stars,3.0
3,5.0,True,A2C1X7H3GJCIGK,B00136J7OA,Positive,This is one of Micheals great songs! I can't s...,26.0
4,NaN,NaN,NaN,NaN,Neutral,besides like this one arsenic well three star,NaN


In [12]:
#drop non-English comments
from langdetect import detect

def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except:
        return False

df_balanced = df_balanced[df_balanced['NewReview'].apply(is_english)]

In [13]:
#checking the values of Sentiment after removing non-English reviews
df_balanced['Sentiment'].value_counts() 

Positive    9446
Neutral     8744
Negative    8167
Name: Sentiment, dtype: int64

In [ ]:
#df_balanced = df_balanced.sample(n=1000, random_state=42)

In [14]:
df_balanced = df_balanced.drop(columns=['verified','overall','reviewerID'])

In [15]:
df_balanced.head()

,asin,Sentiment,NewReview,review_length
0,B009IQ2RSU,Positive,"I like this song, it is worth buying and spend...",25.0
1,B00ZF3SIDI,Positive,Loved this!! Five Stars,4.0
2,B00UB4WZSA,Neutral,good Three Stars,3.0
3,B00136J7OA,Positive,This is one of Micheals great songs! I can't s...,26.0
4,NaN,Neutral,besides like this one arsenic well three star,NaN


## clean text

In [ ]:
# import re
# import string
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer, WordNetLemmatizer

# # Initialize stopwords
# STOPWORDS = set(stopwords.words('english'))

# # Convert NewReview to lowercase
# df_balanced['NewReview'] = df_balanced['NewReview'].str.lower()

# # Remove URLs from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'http\S+', '', x))

# # Remove currency symbols from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'£|\$', '', x))

# # Remove phone numbers from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'^[\(]?[\d]{3}[\)\-]?[\s\.]?[\d]{3}[\-\s\.]?[\d]{4}$', '', x))

# # Remove digits from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'\d+', '', x))

# # Remove email addresses from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'^.+@[^\.].*\.[a-z]{2,}$', '', x))

# # Remove special characters from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'[\'“’]', '', x))

# # Remove stopwords from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: " ".join([word for word in x.split() if word.lower() not in STOPWORDS]))

# # Remove empty values from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: '' if pd.isna(x) or not isinstance(x, str) else x)


In [16]:
from nltk.stem import *

# Initialize stopwords
STOPWORDS = set(stopwords.words('english'))
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define function for text cleaning
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove currency symbols
    text = re.sub(r'£|\$', '', text)

    # Remove phone numbers
    text = re.sub(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove punctuation and special characters except emojis
    #text = re.sub(r'[^\w\s' + emoji.get_emoji_regexp() + ']', '', text)
    text = demoji.replace(text, '')

    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords and handle negation
    negation_words = ['not', 'no', 'never', 'neither', 'nor']
    words = [word if word.lower() in negation_words else word.lower() for word in words]
    words = [word for word in words if word.lower() not in STOPWORDS]
    words = [f'NOT_{words[i+1]}' if (i < len(words)-1 and words[i].lower() in negation_words and words[i+1] not in string.punctuation) else words[i] for i in range(len(words))]
    
    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]

    # Stem words
    words = [stemmer.stem(word) for word in words]

    # Join words back into text
    cleaned_text = ' '.join(words)
    return cleaned_text


In [ ]:
#df_balanced['processed_review'] = df_balanced['NewReview'].apply(clean_text)

In [17]:
from multiprocessing import cpu_count, Pool
pool = Pool(cpu_count())
tasks = df_balanced['NewReview'].tolist()
done = pool.map(clean_text, tasks)  # parallel processing
df_balanced['processed_review'] = done


In [18]:
df_balanced.head()

,asin,Sentiment,NewReview,review_length,processed_review
0,B009IQ2RSU,Positive,"I like this song, it is worth buying and spend...",25.0,"like song , worth buy spend money . instrument..."
1,B00ZF3SIDI,Positive,Loved this!! Five Stars,4.0,love ! ! five star
2,B00UB4WZSA,Neutral,good Three Stars,3.0,good three star
3,B00136J7OA,Positive,This is one of Micheals great songs! I can't s...,26.0,one micheal great song ! ca n't sit still star...
4,NaN,Neutral,besides like this one arsenic well three star,NaN,besid like one arsen well three star


In [19]:
df_balanced = df_balanced.drop(columns=['NewReview'])

In [20]:
df_balanced.head()

,asin,Sentiment,review_length,processed_review
0,B009IQ2RSU,Positive,25.0,"like song , worth buy spend money . instrument..."
1,B00ZF3SIDI,Positive,4.0,love ! ! five star
2,B00UB4WZSA,Neutral,3.0,good three star
3,B00136J7OA,Positive,26.0,one micheal great song ! ca n't sit still star...
4,NaN,Neutral,NaN,besid like one arsen well three star


### save to file, save time

In [8]:
#drop non-English comments
from langdetect import detect

def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except:
        return False

#music_df = music_df[music_df['NewReview'].apply(is_english)]




In [9]:
#music_df.to_json("music_df_en_only.json.gz", orient='records',lines=True)

In [10]:
from multiprocessing import cpu_count, Pool
pool = Pool(cpu_count())
tasks = music_df['NewReview'].tolist()
#done = pool.map(clean_text, tasks)  # parallel processing
#music_df['processed_review'] = done

In [11]:
#music_df = music_df.drop(columns=['NewReview'])

In [12]:

#music_df.to_json("music_df_en_only_cleaned.json.gz", orient='records',lines=True)

In [14]:
music_df = pd.read_json("music_df_en_only_cleaned.json.gz", orient='records',lines=True)


In [15]:
music_df.head()

,reviewerID,asin,helpful,overall,Sentiment,review_length,processed_review
0,A3EBHHCZO6V2A4,5555991584,"[3, 3]",5,Positive,157,'s hard believ `` memori tree '' came 11 year ...
1,AZPWAXJG9OJXV,5555991584,"[0, 0]",5,Positive,105,"clasically-styl introvert album , memori tree ..."
2,A38IRL0X2T4DPF,5555991584,"[2, 2]",5,Positive,88,not_thought thought enya would reach sublim he...
3,A22IK3I6U76GX0,5555991584,"[1, 1]",5,Positive,95,third review irish album write today ( other c...
4,A1AISPOIIHTHXX,5555991584,"[1, 1]",4,Positive,719,"enya , despit success record artist , n't broa..."


In [ ]:
X = pd.DataFrame(df_balanced['processed_review'])
y = df_balanced['Sentiment']

TfidfVectorizer / Naive Bayes

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Count Vectorizer / Naive Bayse

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define CountVectorizer
count_vectorizer = CountVectorizer()

# Define Naive Bayes classifier
nb_classifier = MultinomialNB()

# Combine CountVectorizer and Naive Bayes classifier into a pipeline
pipeline = Pipeline([
    ('vectorizer', count_vectorizer),
    ('classifier', nb_classifier)
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")


In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

#### Logistic Regression

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
model = make_pipeline(
    CountVectorizer(),
    StandardScaler(with_mean=False),
    LogisticRegression(random_state=42)
)

# Fit the model to the training data
model.fit(X_train['processed_review'], y_train)

# Predict the sentiment of the test data
y_pred = model.predict(X_test['processed_review'])

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8743203944872929


In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2283   94   74]
 [ 149 2203  271]
 [  79  327 2429]]
              precision    recall  f1-score   support

    Negative       0.91      0.93      0.92      2451
     Neutral       0.84      0.84      0.84      2623
    Positive       0.88      0.86      0.87      2835

    accuracy                           0.87      7909
   macro avg       0.87      0.88      0.88      7909
weighted avg       0.87      0.87      0.87      7909



#### Gradient Boosting

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('gb', GradientBoostingClassifier())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


Accuracy: 0.831


In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2087  163  201]
 [ 352 1893  378]
 [  23  218 2594]]
              precision    recall  f1-score   support

    Negative       0.85      0.85      0.85      2451
     Neutral       0.83      0.72      0.77      2623
    Positive       0.82      0.91      0.86      2835

    accuracy                           0.83      7909
   macro avg       0.83      0.83      0.83      7909
weighted avg       0.83      0.83      0.83      7909



SVM

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


Accuracy: 0.916


In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2278   94   79]
 [  84 2278  261]
 [  15  134 2686]]
              precision    recall  f1-score   support

    Negative       0.96      0.93      0.94      2451
     Neutral       0.91      0.87      0.89      2623
    Positive       0.89      0.95      0.92      2835

    accuracy                           0.92      7909
   macro avg       0.92      0.92      0.92      7909
weighted avg       0.92      0.92      0.92      7909



In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])


# Define the hyperparameter grid
param_grid = {
    'svm__C': [0.1, 1, 10, 100,200],
    'svm__kernel': ['linear', 'rbf', 'sigmoid','precomputed','poly'],
    'svm__gamma': ['scale', 'auto','float','array-like']
}


# Define the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the pipeline on the training data
grid_search.fit(X_train['processed_review'], y_train)

# Print the best hyperparameters
print("Best parameters: ", grid_search.best_params_)

# Predict on the test data using the best estimator
y_pred = grid_search.best_estimator_.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(C= 100, gamma= 'scale', kernel='rbf'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")

Accuracy: 0.921


In [ ]:
from sklearn.calibration import CalibratedClassifierCV

# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(C= 100, gamma= 'scale', kernel='rbf', probability=True)) # set probability to True
])

# Wrap the pipeline in a calibrated classifier
calibrated_pipeline = CalibratedClassifierCV(pipeline, method='sigmoid', cv=5) # you can choose different calibration methods

# Fit the pipeline on the training data
calibrated_pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = calibrated_pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


##  Recommender -
### inspired by: "Hariri N et el (2011) Context-aware recommendation based on review mining"
### with some modifications, e.g.: using LDA instead of L-LDA, due to we don't have labels in the data etc.

### Load metadata

In [21]:
from keras.utils import get_file

dir1 = get_file(origin="https://raw.githubusercontent.com/shahrzadko/NLPDigitalMusic/main/meta_Digital_Music.json.gz?token=GHSAT0AAAAAAB6LOL42SKHPCCMIVYPOC4REY7AKE2Q")
dir1

import shutil
shutil.move(dir1, "./meta_Digital_Music.json.gz")

42335451/42335451 [==============================] - 0s 0us/step


'./meta_Digital_Music.json.gz'

In [22]:
import ast
import html

path = 'meta_Digital_Music.json.gz'
i = 0
df_dict = {}

search_full = ""

jgz = gzip.open(path, 'rb')
for line in jgz:
    i += 1
    #line, _ = re.subn(r'\'','"',line.decode('utf-8'))
    #d = json.loads(line)
    d = ast.literal_eval(line.decode('utf-8'))
    
    if 'title' not in d or not len(d['title']): continue #remove empty title

    if re.search(r'[<>]',d['title']): continue #remove inviald title that only got html tag
    d['title'] = html.unescape(d['title'])
    
    if d['asin'] in df_dict: continue #remove duplicate items

    df_dict[d['asin']] = d 

mdata_df = pd.DataFrame.from_dict(df_dict, orient='index')
print(i)

279899


In [23]:
mdata_df.shape

(7321, 9)

In [24]:
mdata_df.head()

,asin,title,price,imUrl,related,salesRank,categories,description,brand
5555991584,5555991584,Memory of Trees,9.49,http://ecx.images-amazon.com/images/I/51b5WDjd...,"{'also_bought': ['B000002LRT', 'B000002LRR', '...",{'Music': 939190},"[[CDs & Vinyl, New Age, Celtic New Age], [CDs ...",NaN,NaN
6308051551,6308051551,Don't Drink His Blood,8.91,http://ecx.images-amazon.com/images/I/31LT2n7Q...,NaN,NaN,"[[Digital Music, Alternative Rock, Indie & Lo-...",NEW Combo BLUWAVS CD and FLAC FILE,NaN
7901622466,7901622466,On Fire,11.33,http://ecx.images-amazon.com/images/I/21W29WZw...,"{'also_bought': ['B00000282A', 'B0084O8O9S', '...",{'Music': 58799},"[[CDs & Vinyl, Christian, Rock & Alternative],...",NaN,NaN
B0000000ZW,B0000000ZW,Changing Faces,23.64,http://ecx.images-amazon.com/images/I/51H3Bp-3...,"{'also_bought': ['B00000010Z', 'B0000039Q5', '...",{'Music': 68784},"[[CDs & Vinyl, Dance & Electronic], [CDs & Vin...",,
B00000016W,B00000016W,Pet Sounds,9.49,http://ecx.images-amazon.com/images/I/51pDGkXj...,"{'also_bought': ['B0025KVLTM', 'B00007FOMP', '...",{'Music': 77205},"[[CDs & Vinyl, Classic Rock], [CDs & Vinyl, Po...",NaN,NaN


### Latent Dirichlet Allocation (LDA)

In [25]:
music_df = pd.read_json("music_df_en_only_cleaned.json.gz", orient='records',lines=True)

In [26]:
music_df_s = music_df.sample(n=20000, random_state=42)

In [27]:
music_df.shape

(64630, 7)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(analyzer='word',max_df=1.0)
tfidf_matrix = tfv.fit_transform(music_df_s['processed_review'])
tfidf_matrix.shape

(20000, 52835)

In [29]:
vocab = tfv.get_feature_names_out()
#vocab[:90]

In [30]:
n_topics = 20

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=n_topics,max_iter=20,random_state=42,n_jobs=-1)
topics_of_reviews = lda.fit_transform(tfidf_matrix) # or you can skip this, use lda_model.pickle: uncomment the 2nd one (loading part) below 
topics_of_reviews.shape

(20000, 20)

In [29]:
topic_words = lda.components_

for topic in topic_words:
  st = np.argsort(topic)
  words = np.array(vocab)[st]
  print(words[:-9:-1])

['buffett' 'primu' 'ledisi' 'jimmi' 'claypool' 'margaritavil' 'parrothead'
 'flipper']
['not_object' 'object' 'metric' 'enigma' 'trae' 'souvlaki' 'cretu'
 'boingo']
['tegan' 'pumpkin' 'corgan' 'chevel' 'zeitgeist' 'yusuf' 'norma' 'gino']
['kindl' 'burrel' 'turrentin' 'minaj' 'lissen' 'listern' 'sm' 'hubbi']
['deerhunt' 'afroman' 'krush' 'doggfath' 'yah' 'yester' 'abk' 'sudol']
['demi' 'mane' 'lovato' 'showoff' 'rlj' 'dorham' 'dar' 'skyscrap']
['hurdi' 'gurdi' 'dariu' 'zumba' 'kasey' 'tyte' 'kast' 'burma']
['getz' 'jobim' 'bossa' '65533' 'nova' 'gilberto' 'mingu' 'astrud']
['fogelberg' 'miley' 'boz' 'scagg' 'cohe' 'frou' 'orchid' 'tevin']
['decemberist' 'crane' 'meloy' 'soulif' 'flyleaf' 'colin' 'hazard' 'iso']
['halen' 'van' 'roth' 'shakira' 'eddi' 'vh' 'hagar' 'panama']
['dan' 'steeli' 'fagen' 'aja' '8217' 'becker' '8221' '8220']
['album' 'rap' 'cd' 'song' 'beat' 'good' 'quot' 'like']
['quot' 'album' 'song' 'music' 'one' 'like' 'cd' 'great']
['lyle' 'hdcd' 'totbl' 'skyfal' 'kasabian' 

### k Nearest Neighbors (k-NN)

In [50]:
found, no_mdata = 0,0
for asin, rv in zip(music_df_s['asin'],topics_of_reviews):
    if not asin or asin not in df_dict: no_mdata+=1; continue
    df_dict[asin]['rv_topics'] = rv if 'rv_topics' not in df_dict[asin] else np.vstack((df_dict[asin]['rv_topics'],rv))
    found+=1
found, no_mdata

(49954, 10046)

In [51]:
items_w_rv = dict()
topic_matrix = np.empty((0,n_topics), 'float64')
title_lookup = dict()
asin_lookup = dict()

idx = 0 
for d in df_dict.values():
  if 'rv_topics' in d:
    if len(d['rv_topics'].shape)>1: d['rv_topics'] = np.mean(d['rv_topics'],axis=0)
    topic_matrix = np.vstack((topic_matrix, d['rv_topics']))
    items_w_rv[idx] = d
    title_lookup[d['title'].lower()] = idx
    asin_lookup[d['asin']] = idx
    idx+=1

topic_matrix.shape


(2670, 20)

In [52]:
import pickle
items_to_pickle = (items_w_rv , topic_matrix , title_lookup , asin_lookup)
#with open("lda_model.pickle", 'wb') as pkf:
#  pickle.dump(items_to_pickle, pkf)

In [53]:
## load model option

import pickle
#with open("lda_model.pickle", 'rb') as pkf:
#  (items_w_rv , topic_matrix , title_lookup , asin_lookup) = pickle.load(pkf)

In [54]:
pd.DataFrame.from_dict(items_w_rv, orient='index')

,asin,title,price,imUrl,related,salesRank,categories,rv_topics,brand,description
0,5555991584,Memory of Trees,9.49,http://ecx.images-amazon.com/images/I/51b5WDjd...,"{'also_bought': ['B000002LRT', 'B000002LRR', '...",{'Music': 939190},"[[CDs & Vinyl, New Age, Celtic New Age], [CDs ...","[0.005944301246778397, 0.005724376118965009, 0...",NaN,NaN
1,B0000000ZW,Changing Faces,23.64,http://ecx.images-amazon.com/images/I/51H3Bp-3...,"{'also_bought': ['B00000010Z', 'B0000039Q5', '...",{'Music': 68784},"[[CDs & Vinyl, Dance & Electronic], [CDs & Vin...","[0.007947564104963068, 0.007947564145385768, 0...",,
2,B00000016W,Pet Sounds,9.49,http://ecx.images-amazon.com/images/I/51pDGkXj...,"{'also_bought': ['B0025KVLTM', 'B00007FOMP', '...",{'Music': 77205},"[[CDs & Vinyl, Classic Rock], [CDs & Vinyl, Po...","[0.008476647700033773, 0.007341975014550384, 0...",NaN,NaN
3,B00000016T,Cars,14.63,http://ecx.images-amazon.com/images/I/51pmW%2B...,"{'also_bought': ['B000002GWX', 'B000002GXD', '...",{'Music': 458151},"[[CDs & Vinyl, Alternative Rock, New Wave & Po...","[0.00804670266352813, 0.009927344553832798, 0....",NaN,NaN
4,B00000017R,Jazz Samba,8.99,http://ecx.images-amazon.com/images/I/51kAU0Ty...,"{'also_bought': ['B0000046WB', 'B0000047CX', '...",{'Music': 561174},"[[CDs & Vinyl, Jazz, Bebop], [CDs & Vinyl, Jaz...","[0.007089352362394035, 0.006163632120142277, 0...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2665,B000LXHGBC,Back Numbers,11.88,http://ecx.images-amazon.com/images/I/51N%2ByT...,"{'also_bought': ['B00005YN6S', 'B00HVF1344', '...",{'Music': 124106},"[[CDs & Vinyl, Alternative Rock, Indie & Lo-Fi...","[0.006629234050171659, 0.006629234067383804, 0...",NaN,The thoroughly excellent sophomore release by ...
2666,B000M5B6GK,All of a Sudden I Miss Everyone,9.49,http://ecx.images-amazon.com/images/I/618p7PKX...,"{'also_bought': ['B0006Q2TPC', 'B0000DJYMF', '...",{'Music': 640730},"[[CDs & Vinyl, Alternative Rock, Indie & Lo-Fi...","[0.006477516765231249, 0.005191601906141134, 0...",NaN,Sometimes Explosions in the Sky start with a w...
2667,B000MGBTIE,Alpha,13.83,http://ecx.images-amazon.com/images/I/51NLyLG1...,"{'also_bought': ['B000B8QF6O', 'B0013K1ALY', '...",{'Music': 5783},"[[CDs & Vinyl, Alternative Rock, Goth & Indust...","[0.00709600084750042, 0.005779527698819136, 0....",Alpha,"Alpha, the latest outing from this veteran act..."
2668,B000MGUZ9I,Undiscovered,10.88,http://ecx.images-amazon.com/images/I/51SAKEc0...,"{'also_bought': ['B001EOQTV0', 'B005ILWOZA', '...",{'Music': 8872},"[[CDs & Vinyl, Alternative Rock], [CDs & Vinyl...","[0.005569142681427098, 0.005569142662887866, 0...",NaN,"Young James Morrison has ""success story"" writt..."


In [55]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=11,n_jobs=-1)
knn.fit(topic_matrix)


NearestNeighbors(n_jobs=-1, n_neighbors=11)

In [56]:
#knn.kneighbors([topic_matrix[37]],return_distance=False)

In [57]:
def recommand_by_title(title):
  if title not in title_lookup: 
    print("\nSorry, we don’t have recommendations for", title)
    return
  print(f"\nRecommandations for - {title}:\n")

  idx = title_lookup[title]
  recom_list = knn.kneighbors([topic_matrix[idx]],return_distance=False)[0][1:]
  for ridx in recom_list:
    print(items_w_rv[ridx]['title'])


### recommand loop 

In [58]:
print("\nInput a title (use 'exit' to exit): ", end='')
user_input = input().lower()

while(user_input and user_input != 'exit'):
  recommand_by_title(user_input)

  print("\nInput a title (use 'exit' to exit): ", end='')
  user_input = input().lower()


Input a title (use 'exit' to exit): undiscovered

Recommandations for - undiscovered:

Talk Talk Talk
Hearts and Bones
Paradise & Lunch
Nested
Both Sides
Chinese Wall
Living in Clip
Brand New Me
Everglow
Non-Stop Erotic Cabaret

Input a title (use 'exit' to exit): Hearts and Bones

Recommandations for - hearts and bones:

Nested
Birdy
Undiscovered
Reg Strikes Back
Now in a Minute
Oh No
Ambrosia
Affection
Stay With Me
Welcome Home

Input a title (use 'exit' to exit): exit
